# Welcome to better ebird API

this is a fun lab that is 

Install and import the following packages 

In [2]:
import pandas as pd
import geopandas as gpd
import requests
import json
import folium


/Users/laurenfitzgerald/Desktop/geog464/venv3.11.1/lib/python3.11/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/8d/r4tft4n531l4mrzp4lvnytsm0000gn/T/ipykernel_86294/3642698031.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEO

In [74]:
# Define all necessary API and CX keys
# if you dont have access to google API that is okay, you just wont be able to use the functions coded with 'GAPI'
google_apikey = "paste your google api key here"
google_CX = "paste your google CX here"
ebird_api_key = 'your ebird api key'

First import the csv file with all the information on world birds.

In [ ]:
# read the following csv file. this is necessary for the following fuctions: com_to_sci, sci_to_com, list_types
species_data = pd.read_csv("PFW-species-translation-table.csv")
species_data

,species_code,alt_full_spp_code,n_locations,scientific_name,american_english_name,nom_francais_canadien,nombre_espanol_mexicano,taxonomy_version,taxonomic_sort_order
0,grerhe1,NaN,1,Rhea americana,Greater Rhea,Nandou d'Amérique,Ñandú Común,2021.01,8
1,gretin1,NaN,3,Tinamus major,Great Tinamou,Grand Tinamou,Tinamú Mayor,2021.01,39
2,wfwduc1,NaN,1,Dendrocygna viduata,White-faced Whistling-Duck,Dendrocygne veuf,Pijije Cariblanco,2021.01,219
3,bbwduc,NaN,65,Dendrocygna autumnalis,Black-bellied Whistling-Duck,Dendrocygne à ventre noir,Pijije Alas Blancas,2021.01,221
4,fuwduc,NaN,2,Dendrocygna bicolor,Fulvous Whistling-Duck,Dendrocygne fauve,Pijije Canelo,2021.01,226
...,...,...,...,...,...,...,...,...,...
1067,whcsee1,NaN,1,Sporophila morelleti,Morelet's Seedeater,Sporophile de Morelet,Semillero de Collar,2021.01,34424
1068,whcsee2,NaN,1,Sporophila torqueola,Cinnamon-rumped Seedeater,Sporophile à col blanc,Semillero Rabadilla Canela,2021.01,34427
1069,whcsee,NaN,3,Sporophila morelleti/torqueola,Morelet's/Cinnamon-rumped Seedeater,Sporophile de Morelet ou S. à col blanc,Semillero Semillero de Collar/Rabadilla Canela,2021.01,34430
1070,grasal2,NaN,1,Saltator grandis,Cinnamon-bellied Saltator,Saltator du Mexique,Saltador Gris Mesoamericano,2021.01,34654


### Now that we have the data we can define and run functions based off of it!

This first function can be used to retive the scientific name of the bird using the common name as the input

In [24]:
def com_to_sci(common_name):
    # make it not case sensitve 
    common_name = common_name.lower()
    scientific_name = species_data.loc[species_data['american_english_name'].str.lower() == common_name, 'scientific_name'].values[0]
    return scientific_name
    

In [25]:
# test it out by inputting any common bird name into the code such as "house wren"
com_to_sci('house wren')

'Troglodytes aedon'

In [26]:
# try it yourself! (remove # from the next line)
# com_to_sci("instert common bird name here")

This next function is the reverse of **com_to_sci**, It can be used to retive the common name of the bird using the scientific name as the input

In [27]:
def sci_to_com(scientific_name):
    scientific_name = scientific_name.lower()
    common_name = species_data.loc[species_data['scientific_name'].str.lower() == scientific_name, 'american_english_name'].values[0]
    return(common_name)


In [28]:
# test it out by inputting any scientific bird name into the code such as "Troglodytes aedon"
sci_to_com('Troglodytes aedon')

'House Wren'

In [29]:
# try it yourself! (remove # from the next line)
# sci_to_com("instert common bird name here")

The third function **list_types** finds matching names and returns a list of all the speices that contain a specified word such as "duck" or "wren" 

In [30]:
def list_types(sub_species):
    matching_names = species_data[species_data['american_english_name'].str.contains(sub_species, case=False)]['american_english_name'].tolist()
    return matching_names

In [65]:
# test it out by inputting any type of bird such as robin
list_types("hummingbird")

['Ruby-topaz Hummingbird',
 "Rivoli's Hummingbird",
 'Lucifer Hummingbird',
 'Ruby-throated Hummingbird',
 'Black-chinned Hummingbird',
 'Ruby-throated/Black-chinned Hummingbird',
 "Anna's Hummingbird",
 "Costa's Hummingbird",
 "Anna's x Costa's Hummingbird (hybrid)",
 "Anna's/Costa's Hummingbird",
 'Calliope Hummingbird',
 "Anna's x Calliope Hummingbird (hybrid)",
 'Rufous Hummingbird',
 "Allen's Hummingbird",
 "Rufous/Allen's Hummingbird",
 'Broad-tailed Hummingbird',
 'Broad-billed Hummingbird',
 'White-eared Hummingbird',
 'Violet-crowned Hummingbird',
 'Berylline Hummingbird',
 'Cinnamon Hummingbird',
 'Buff-bellied Hummingbird',
 'Rufous-tailed Hummingbird',
 'White-throated Hummingbird',
 'hummingbird sp.']

## Using Beautiful Soup to scrape websites 

the following functions use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) this python libary is used pulls information from html websites.

To use beauiful soup, you must locate the infomration you want to pull using "inspect" to find the div name

In [33]:
# if its not already installed do the following:
# !pip install bs4

In [34]:
#import the following:
import bs4
from bs4 import BeautifulSoup
import requests

The first function will pull information from [The Audubon Bird Guide](https://www.audubon.org/bird-guide). This guide has access to all the birds in North America

The website url is set up like this 
https://www.audubon.org/field-guide/bird/aberts-towhee

As you can see from the link above, the common name of the bird is at the end and any spaces in the name are replaced with a '-'. 

In [35]:
def com_des(common_name):
    # we must replace all spaces with a '-' for the function to work
    common_name = common_name.replace(" ", "-")
    # we know the common name goes at the end based off of exploring the website
    url = f"https://www.audubon.org/field-guide/bird/{common_name}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # by inspecting the website we know where the desription of the bird is located
    bird_div = soup.find("div", class_="bird_discussion")
    if bird_div is not None:
        return bird_div.text.strip()
    else:
        return "Bird not found on Audubon Bird Guide. The guide only has access to birds of North America. If the bird is in North America, check spelling"


In [44]:
# test it out 
#com_des("mallard")

What if we want to find a bird out side of North America? 

The following function pulls information from [picturebirdai.com/](https://picturebirdai.com/) this website has access to all the birds of the world! 

The website url is set up like this: https://picturebirdai.com/wiki/Amazilia_yucatanensis.html

As we can see from the link above, the scientific name is at the and and any space in the name are replace with a "_"

In [43]:
def sci_des(scientific_name):
    # we must replace all spaces with a '_' for the function to work
    scientific_name = scientific_name.replace(" ", "_")
     # we know the scientific name is inputted at the end and followed by a ".html" by exploring the website 
    url = f"https://picturebirdai.com/wiki/{scientific_name}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # by inspecting the website we know where the desription of the bird is located
    bird_div = soup.find("div", {"id": "description-content"})
    if bird_div is not None:
        field_value_div = bird_div.find("div", {"class": "field_value_description"})
        if field_value_div is not None:
            div = field_value_div.find("div")
            if div is not None:
                return div.text.strip()
            return "Bird not found. Make sure you are inputting the proper scientific name. Use function com_to_sci to get the proper name"


In [46]:
# test it out!
# if you  dont know the scientific name of the bird you want use the function com_to_sci 
# example
#com_des("Anas platyrhynchos")

Lets combine these functions! Lets define a function called **bird_description** which will take the common name as input and search [The Audubon Bird Guide](https://www.audubon.org/bird-guide), if nothing is found it will then search [picturebirdai.com](https://picturebirdai.com/). 

This function will use **com_to_sci** to convert the common name in order it to be searched in **picturebirdai.com** if it isnt found in **The Audubon Bird Guide**

In [47]:
def bird_description(common_name):
    common_name1 = common_name.replace(" ", "-")
    url = f"https://www.audubon.org/field-guide/bird/{common_name1}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    bird_div = soup.find("div", class_="bird_discussion")
    if bird_div is not None:
        return bird_div.text.strip()
    else:
        scientific_name = com_to_sci(common_name)
        if scientific_name is not None:
            scientific_name1 = scientific_name.replace(" ", "_")
            url = f"https://picturebirdai.com/wiki/{scientific_name1}.html"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            bird_div = soup.find("div", {"id": "description-content"})
            if bird_div is not None:
                field_value_div = bird_div.find("div", {"class": "field_value_description"})
                if field_value_div is not None:
                    div = field_value_div.find("div")
                    if div is not None:
                        return div.text.strip()
            return "Bird not found. Make sure you are inputting the proper scientific name. Use function com_to_sci to get the proper name"
        else:
            return "Bird not found"

In [66]:
# test it out!
bird_description("Allen's Hummingbird")

"Allen's hummingbird (Selasphorus sasin) is a species of hummingbird. It is a small bird, with mature adults reaching only 3 to 3.5 in (76 to 89 mm) in length. The male has a green back and forehead, with rust-colored (rufous) flanks, rump, and tail. The male's throat is an iridescent orange-red. The female and immature Allen's hummingbirds are similarly colored, but lack the iridescent throat patch, instead having a series of speckles on their throats. Females are mostly green, featuring rufous color only on the tail, which also has white tips. Immature Allen's hummingbirds are so similar to the female rufous hummingbird, the two are almost indistinguishable in the field. Both species' breeding seasons and ranges are common factors used to differentiate between the two species in a particular geographical area."

# Explore the ebird API!

This function retieves the data from the ebird API about a specifed species using the scientfic name as the input

In [62]:
def retrieve_species_data(scientific_name):
    url = "https://api.ebird.org/v2/ref/taxonomy/ebird"
    params = {
        "fmt": "json",
        "locale": "en",
        "species": scientific_name
    }
    headers = {
        "X-eBirdApiToken": ebird_api_key
    }
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    if data:
        return data[0]
    else:
        return None

# Retiving Images!

In [ ]:
#Import the following 
import IPython 
from IPython.display import Image

there are two ways to Retieve images:
1. The first way uses BeautifulSoup to find the image within an website 
2. The second way uses a Google API to do a google image search

### First way: Using BeautifulSoup and Wikipedia

In [48]:
def wiki_pic(common_name):
    # Make a GET request to the Wikipedia page
    url = f"https://en.wikipedia.org/wiki/{common_name}"
    common_name = common_name.replace(" ", "_")
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    # Find the infobox table with either class name
    infobox = soup.find("table", class_=["infobox biota", "taxobox"])
    # if there is an infobox find image
    if infobox:
        # Find the image link within the infobox table
        image = infobox.find("img")
        # Get the source URL of the image
        image_url = image["src"]
        # Make a GET request to the image URL
        response = requests.get("https:" + image_url)
        picture_url = ("https:" + image_url)
    # if no infobox they are usually in a div called thmbinner
    else:
        picbox = soup.find("div", class_=["thumbinner"])
        if picbox:
            # Find the first image in the thumbinner div
            image2 = picbox.find("img")
            # Get the source URL of the image
            image_url = image2["src"]
            # Make a GET request to the image URL
            response = requests.get("https:" + image_url)
            picture_url = ("https:" + image_url)
        else:
            return "image not found"  
    #return(picture_url)
    display(Image(url=picture_url))

In [68]:
wiki_pic("Allen's Hummingbird")

### Second way: Using Google Image Search

In [ ]:
# import the following 
from google_images_search import GoogleImagesSearch

In [51]:
#First define your google image API Key and CX Key 
google_apikey = "your google apikey here "
google_CX = "your google cx here" 

In [52]:
def google_API_pic(common_name):
    # Initialize GoogleImagesSearch object
    gis = GoogleImagesSearch(google_apikey, google_CX)
    # Set search parameters
    _search_params = {
        # + bird is added to ensure that only bird images get retieved or else you would retieve TV images if merlin is search or sports team logos etc
        "q": common_name + " bird",
        "imgSize": "large",
        "num": 1
    }
    # Search for images
    gis.search(search_params=_search_params)
    # Get image URL
    image_url = gis.results()[0].url
    pic = requests.get(image_url)
    # Display image in notebook
    return Image(pic.content)

# Encode the images for mapping! 


We are soon going to define some mapping functions however, we will want to use these functions to import the images of the birds into the map markers/popups. 

In order to do so we must encode the images. This will use [base64](https://docs.python.org/3/library/base64.html)

In [53]:
#import the following 
import base64

## For Wiki images 

In [69]:
def wiki_pic_map(common_name):
    # Make a GET request to the Wikipedia page
    url = f"https://en.wikipedia.org/wiki/{common_name}"
    common_name = common_name.replace(" ", "_")
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    # Find the infobox table with either class name
    infobox = soup.find("table", class_=["infobox biota", "taxobox"])
    # Find the first image in the infobox table, if it exists
    if infobox:
        # Find the image link within the infobox table
        image = infobox.find("img")
        # Get the URL of the image
        image_url = image["src"]
        response = requests.get("https:" + image_url)
        # Encode the image content as base64
        image_data = base64.b64encode(response.content).decode()
        # Returnimage data
        return image_data
    # If there is no image in the infobox table, look for the first image in the thumbinner div
    else:
        picbox = soup.find("div", class_=["thumbinner"])
        if picbox:
            # Find the first image in the thumbinner div
            image2 = picbox.find("img")
            # Get the URL of the image
            image_url = image2["src"]
            response = requests.get("https:" + image_url)
            # Encode the image content as base64
            image_data = base64.b64encode(response.content).decode()
            # Return image data
            return image_data
        # If no image is found, return None
        else:
            return None 

## For Google images 

In [70]:
def google_API_pic4map(common_name):
    # Initialize GoogleImagesSearch object
    gis = GoogleImagesSearch(google_apikey, google_CX)

    # Set search parameters
    _search_params = {
        "q": common_name + " bird",
        "imgSize": "large",
        "num": 1
    }

    # Search for images
    gis.search(search_params=_search_params)
    image_data = requests.get(gis.results()[0].url).content
    # Encode image data as base64
    image_base64 = base64.b64encode(image_data).decode()
    
    return image_base64

# Lets Map!

For our maps we will use some of the functions defined earlier in this lab. 

- **bird_description** will be used to pull a descrption of the bird for the marker popup 
- **wiki_pic_map** or **google_API_pic4map** will be used to retieve a picture of the mapped bird for the marker popup

### Our first function will find ***nearby birds*** 

- this function takes the coordinates of any location and a radius distance

In [75]:
def nearby_observations(lat, lng, distance_km):
    # Define the API endpoint URL
    #dis = 15km 
    url = f'https://api.ebird.org/v2/data/obs/geo/recent?lat={lat}&lng={lng}&dist={distance_km}&cat=species&fmt=json'

    #API inp
    headers = {'X-eBirdApiToken': ebird_api_key}
    response = requests.get(url, headers=headers)

    data = response.json()
    
    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    # Loop through the data and add markers for each bird sighting
    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        popup_text = f"<b>{bird_name}</b><br>{sci_name}</br><br>Date:{obs_date}</br>"
            
 # Get bird description
        bird_description_text = bird_description(bird_name)
 # this function defaults to using wiki_pic_map however it can be replace with google_API_pic4map. 
        #google_API_pic4map is better if the user doesnt have a limit in querys as it is more reliable 
        # to use the function replace the next line of code with " bird_image_data = google_API_pic4map(bird_name)
        bird_image_data = wiki_pic_map(bird_name)
        
        # Create custom HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}" style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map




In [ ]:
nearby_observations('45.977083', '-66.672496', '10')

### Our next function will find ***nearby sighting of a specific species***

- this function takes the coordinates of any location and the birds scientific name 
- it has a default radius of 10km

In [59]:
def nearby_bird(lat, lng, sci_name):
    # Define the API endpoint URL
    #dis = 10km
    url = f'https://api.ebird.org/v2/data/obs/geo/recent?lat={lat}&lng={lng}&dist=10&sci={sci_name}&fmt=json'

    headers = {'X-eBirdApiToken': ebird_api_key}
    response = requests.get(url, headers=headers)
    data = response.json()

    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        
        # Get bird description
        bird_description_text = bird_description(bird_name)
         # this function defaults to using wiki_pic_map however it can be replace with google_API_pic4map. 
        #google_API_pic4map is better if the user doesnt have a limit in querys as it is more reliable 
        # to use the function replace the next line of code with " bird_image_data = google_API_pic4map(bird_name)
        bird_image_data = wiki_pic_map(bird_name)
        
        # Create custom HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}"style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map

### Our next function will find ***nearby sighting of notable species***
- this function takes the coordinates of any location and a specified radius in km

In [60]:
def notable_bird(lat, lng, distance_km):
    # Define the API endpoint URL
  
    url =f'https://api.ebird.org/v2/data/obs/geo/recent/notable?lat={lat}&lng={lng}&dist={distance_km}&fmt=json'

    # Add API key to the request headers
    headers = {'X-eBirdApiToken': ebird_api_key}

    # Send GET request to API endpoint
    response = requests.get(url, headers=headers)

    # Convert response to JSON format
    data = response.json()

    # Create a map centered at the inputted lat/lng
    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        
        # Get bird description
        bird_description_text = bird_description(bird_name)
        #if bird_description_text == "Bird not found on Audubon Bird Guide. The guide only has access to birds of North America. If the bird is in North America, check spelling":
          #  bird_description_text = sci_des(sci_name)
        bird_image_data = wiki_pic_map(bird_name)
        
        # Create custom HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}"style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map


### Our last mapping function will find ***nearby birding hotspots***

- Hotspots represent a set of public locations that people regularly visit for birding, regardless of how amazing they are for birds. The primary requirement of a Hotspot is that it is publicly accessible
- this function is great for those wondering where others frequently birdwatch at

In [61]:
def get_hotspot(lat, lng):
    url = f'https://api.ebird.org/v2/ref/hotspot/geo?lat={lat}&lng={lng}&dist=5&fmt=json'
    headers = {'X-eBirdApiToken': '<your_eBird_API_token_here>'}
    response = requests.get(url, headers=headers)
    data = response.json()
    bird_map = folium.Map(location=[lat, lng], zoom_start=13)

    for obs in data:
        loc_id = obs['locId']
        #then use the api to go find the hotspot information so it can be put into the markers 
        hotspot_url = f"https://api.ebird.org/v2/ref/hotspot/info/{loc_id}"
        hotspot_response = requests.get(hotspot_url, headers=headers)
        hotspot_data = hotspot_response.json()
        if hotspot_response.status_code == 200:
            # Create popup text with hotspot info
            popup_text = f"<b> Name: </b><br>{hotspot_data['name']}</br>" \
                         f"<b>Region:</b><br>{hotspot_data['subnational2Name']}</br>" \
                         f"<b>Province/State:</b><br>{hotspot_data['subnational1Name']}</br>" \
                         f"<b>Country:</b><br>{hotspot_data['countryCode']}</br>" \
                         f"<b>Latitude & Longitude:</b><br>{hotspot_data['latitude']}, {hotspot_data['longitude']}</br>"\
            
        else:
            popup_text = "Error getting hotspot info"
            location = obs['lat'], obs['lng']
            marker = folium.Marker(location=location, popup=popup_text)
            marker.add_to(bird_map)

    return bird_map